In [1]:
from dzdy import *

par_script = """
    transmission_rate = 1.5
    rec_rate ~ triangle(0.1, 0.2, 0.3)
    beta ~ exp(transmission_rate)
    gamma ~ exp(rec_rate)
    Die ~ exp(0.02)
"""

dc_ctbn_script = '''
    Name = SIR_bn
    Type = CTBN
    
    life[Alive | Dead]
    sir[S | I | R]
    
    Alive{life:Alive}
    Dead{life:Dead}
    Inf{life:Alive, sir:I}
    Rec{life:Alive, sir:R}
    Sus{life:Alive, sir:S}

    Die -> Dead # from transition Die to state Dead by distribution Die
    Sus -- Infect(beta) -> Inf 
    Inf -- Recov(gamma) -> Rec
    
    Alive -- Die # from state Alive to transition Die
'''

da = DirectorABM()
da.read_pcore('pSIR', par_script)
da.read_dcore(dc_ctbn_script)
cfd = da.new_abm('ABM_SIR', 'pSIR', 'SIR_bn')
cfd.set_observations(states=['Sus', 'Inf', 'Rec'], 
                      behaviours=['transmission'])
cfd.add_fill_up('Distribution', Name = 'BMI', Distribution = 'norm(23,5)')

PCore pSIR added
Dcore SIR_bn added


In [10]:
mod_src = da.generate_abm('ABM_SIR')
mod_src.initialise(0, {'Sus': 4, 'Inf': 1})
mod_src.Pop.Agents

OrderedDict([('Ag1', ID: Ag1, BMI: 23.7187903804, State: Sus),
             ('Ag2', ID: Ag2, BMI: 29.3691911666, State: Sus),
             ('Ag3', ID: Ag3, BMI: 23.331165766, State: Sus),
             ('Ag4', ID: Ag4, BMI: 25.2256896284, State: Sus),
             ('Ag5', ID: Ag5, BMI: 19.7870290976, State: Inf)])

In [11]:
for ag in mod_src.Pop.Agents.values():
    print(ag.next)

Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.1569898597976952
Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.09563379633611281
Tr(Name: Infect, To: Inf, By: exp(1.5)): 1.6196863064768308
Tr(Name: Infect, To: Inf, By: exp(1.5)): 2.0417956172956755
Tr(Name: Recov, To: Rec, By: exp(0.197163119925)): 0.622109860854025


In [ ]:
def copy_agent(ag_src, dc, tr_ch=None):
    if not tr_ch:
        tr_ch = dc.Transitions
    ag_new = Agent(ag_src.Name, dc_new[ag_src.State.Name])

    if not tr_ch:
        return ag_new
    
    for tr, tte in ag_src.Trans.items():
        if tr.Name in tr_ch:
            continue
        ag_new.Trans[dc_new.Transitions[tr.Name]] = tte
    ag_new.Info.update(ag_src.Info)  
    return ag_new

In [36]:
mc = 'ABM_SIR'
tr_tte = True
mod_new = da.generate_abm(mc)
time_copy = mod_src.TimeEnd if mod_src.TimeEnd else 0
pc_new = mod_new.PCore
dc_new = mod_new.DCore
trs = da.get_dcore(da.MCores[mc].TargetedDCore).Transitions
if tr_tte:
    _, ds = pc_new.difference(mod_src.PCore)
    tr_ch = [k for k, v in trs.items() if v['Dist'] in ds]
    for k, v in mod_src.Pop.Agents.items():
        mod_new.Pop.Agents[k] = copy_agent(v, dc_new, tr_ch)
else:
    for k, v in mod_src.Pop.Agents.items():
        mod_new.Pop.Agents[k] = copy_agent(v, dc_new)

mod_new.Pop.Eve.Last = mod_src.Pop.Eve.Last
# network
        
# mod

for ag in mod_new.agents:
        ag.update(time_copy)

for ag in mod_src.Pop.Agents.values():
    print(ag.next)
print()
for ag in mod_new.Pop.Agents.values():
    print(ag.next)

Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.1569898597976952
Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.09563379633611281
Tr(Name: Infect, To: Inf, By: exp(1.5)): 1.6196863064768308
Tr(Name: Infect, To: Inf, By: exp(1.5)): 2.0417956172956755
Tr(Name: Recov, To: Rec, By: exp(0.197163119925)): 0.622109860854025

Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.1569898597976952
Tr(Name: Infect, To: Inf, By: exp(1.5)): 0.09563379633611281
Tr(Name: Infect, To: Inf, By: exp(1.5)): 1.6196863064768308
Tr(Name: Infect, To: Inf, By: exp(1.5)): 2.0417956172956755
Tr(Name: Recov, To: Rec, By: exp(0.230533410121)): 0.022100606738555562
